In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train/255.0, x_test/255.0
print("x_train.shape:",x_train.shape)

In [ ]:
#As the data is only 2D than is why for CNN we need to make it 3D channels
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train.shape:",x_train.shape)

In [ ]:
# K is the number of classes|| number of unique elements
K = len(set(y_train))
print("K is the number of classes = ",K)

In [ ]:
# Building the CNN model. This model is for the single input and a single output 
i = Input(shape = x_train[0].shape)
x = Conv2D(32, (3,3), padding= 'same', activation = 'relu')(i)
x = BatchNormalization()(x)
x = Conv2D(32, (3,3), padding= 'same', activation = 'relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)
x = Conv2D(64, (3,3), padding= 'same', activation = 'relu')(x)
x = BatchNormalization()(x)
x = Conv2D(64, (3,3), padding= 'same', activation = 'relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)
x = Conv2D(128, (3,3), padding= 'same', activation = 'relu')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3,3), padding= 'same', activation = 'relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)
x = Flatten()(x)
x = Dropout(0.2)(x)
x = Dense(1024, activation = 'relu')(x)
x = Dropout(0.2)(x)
x = Dense(K, activation = 'softmax')(x)
model = Model(i,x)

In [ ]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

r = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs= 50)

In [ ]:
# Fit the data with Augmentation 
# If I run this after calling the previous model.fit(), it will continue training where it left off

batch_size = 32
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(width_shift_range=0.1, height_shift_range = 0.1, horizontal_flip = True)
train_generator = data_generator.flow(x_train,y_train,batch_size)
steps_per_epoch = x_train.shape[0]//batch_size
r = model.fit_generator(train_generator,validation_data = (x_test,y_test), steps_per_epoch = steps_per_epoch, epochs= 50)

In [ ]:
#Plotting the loss per iteration 
plt.plot(r.history['loss'], label = 'loss')
plt.plot(r.history['val_loss'], label = 'val_loss')
plt.legend()

In [ ]:
# Plotting accuracy per iteration 
plt.plot(r.history['accuracy'], label = 'acc')
plt.plot(r.history['val_accuracy'], label = 'val_loss')
plt.legend()

In [ ]:
from prompt_toolkit.layout import HorizontalAlign
# Plotting the confusion matrix
from sklearn.metrics import confusion_matrix
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize = False,
                          title = 'Confusion Matrix',
                          cmap = plt.cm.Blues):
  
  if normalize:
    cm = cm.astype('float')/ cm.sum(axis = 1)[:, np.newaxis]
    print("Normalized confusion matix")
  else:
    print('Confusion matrix, without normalization')
  print(cm)

  plt.imshow(cm, interpolation = 'nearest', cmap= cmap)
  plt.title(title)
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation = 45)
  plt.yticks(tick_marks, classes)

  fmt = '.2f' if normalize else 'd'
  thresh = cm.max()/2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, format(cm[i,j],fmt),
             horizontalalignment = 'center',
             color = 'white' if cm[i,j]> thresh else 'black')
  plt.tight_layout()
  plt.ylabel('True Label')
  plt.xlabel("Predicted Label")
  plt.show()

p_test = model.predict(x_test).argmax(axis=1)
cm = confusion_matrix(y_test,p_test)
plot_confusion_matrix(cm, list(range(10)))

In [ ]:
# From the confusion matrix we observe that the model is pretty much confused with 
# to differentiate between [0]= 'T-shirt/top and [6]='shirt' which is 123 and make sense
labels = '''T-shirt/top
Trouser
Pullover
Dress
Coat
Sandal
Shirt
Sneaker
Bag
Ankle boot'''.split()
misclassified_idx = np.where(p_test != y_test)[0]
i = np.random.choice(misclassified_idx)
plt.imshow(x_test[i].reshape(28,28), cmap = 'gray')
plt.title("True Label: %s Predicted Label: %s" %(labels[y_test[i]], labels[p_test[i]]));


In [ ]:
model.summary()